<a href="https://colab.research.google.com/github/nkd98/mip-automation-project/blob/main/MIP_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# MIP Data Analysis
# Author: Niruj Deka
# Last Updated: 31-Dec-24

# Introduction and Setup

In [2]:
# prompt: install all the neccesary packages to run this notebook

!pip install pandas


In [3]:
# importing pacakges
import pandas as pd


In [4]:
# provide access to Google Drive. Authorise Google Drive when prompted
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# check currect working directory
!pwd

## Please ensure that the folder you will be working with is in your Drive.
## If it is a shared folder, you need to move it to your Drive.

# list the files in the directory : change it your folder
!ls /content/drive/MyDrive/MIP

/content
school_list.gsheet  school_list.xlsx  School_List.xlsx	Status_Report_30Dec.csv


# Load Datasets.
Requirements: 1) MIP Data, 2) List of middle Schools.
Please ensure that both datasets is in the same folder.

In [6]:
######## Read MIP data

# Assuming the file is named 'Status_Report_30Dec.csv' and is located in the folder.
# Replace 'Status_Report_30Dec.csv' and the folder path if necessary.

mip_file_path = '/content/drive/MyDrive/MIP/Status_Report_30Dec.csv'  # Update with the correct path

# read the data
mip_data = pd.read_csv(mip_file_path)

# check the columns
mip_data.columns

Index(['UUID', 'User Type', 'User sub type', 'Declared State', 'District',
       'Block', 'School Name', 'School ID', 'Declared Board', 'Org Name',
       'Program Name', 'Program ID', 'Project ID', 'Project Title',
       'Project Objective', 'Project start date of the user',
       'Project completion date of the user', 'Project Duration',
       'Project last Synced date', 'Project Status', 'Certificate Status'],
      dtype='object')

In [7]:
######### Read the School Data: List of all middle-schools

# Assuming the xlsx file is named 'School_List.xlsx' and is located in the same directory.
# Replace 'School_List.xlsx' with the actual file name.
sch_file_path = '/content/drive/MyDrive/MIP/school_list.xlsx'  # Update with the correct path

# Read data from
school_data = pd.read_excel(sch_file_path)

#print(school_data.head()) # print the first 5 rows to check

# Keep only necessary columns
school_data = school_data[['DISTRICT NAME', 'BLOCK NAME','UDISE+ SCHOOL CODE', 'SCHOOL NAME']]
#print(school_data.head()) # print the first 5 rows to check

# Rename the column from 'UDISE+ SCHOOL CODE' to 'School ID'
school_data = school_data.rename(columns={'UDISE+ SCHOOL CODE': 'School ID'})
print(school_data.head()) # print the first 5 rows to check


  DISTRICT NAME BLOCK NAME    School ID           SCHOOL NAME
0        ARARIA     ARARIA  10071200101          UMS ARARBARI
1        ARARIA     ARARIA  10071200301     UMS KASAILA  GIRL
2        ARARIA     ARARIA  10071200401              MS JAMUA
3        ARARIA     ARARIA  10071200402  U.M.S JAMUA KHAMGARA
4        ARARIA     ARARIA  10071200403    UMS JAMUA GHARWARI


# Cleaning and Merging of Datasets

In [8]:

# Check for duplicates in the 'School ID' column of MIP Data
num_duplicates = mip_data['School ID'].duplicated(keep=False).sum()

if num_duplicates > 0:
    print(f"There are {num_duplicates} duplicates in the 'School ID' column.")
else:
    print("There are no duplicates in the 'School ID' column.")

# Create a new variable which takes value completed if value of projectstatus is completed, takes value started if not
mip_data['MIP_Status'] = mip_data['Project Status'].apply(lambda x: 'submitted' if x == 'submitted' else 'started')


There are 9365 duplicates in the 'School ID' column.


In [9]:
# If a School ID has multiple MIP_Status values, it keeps the row with 'completed' if available;
# otherwise, it keeps the first available status for that school.

# Step 1: Sort the data by 'School ID' and 'MIP_Status' (alphabetical order)
mip_data_sorted = mip_data.sort_values(by=['School ID', 'MIP_Status'], ascending=[True, True])

# Step 2: Drop duplicates based on 'School ID', keeping the first row (which will be the 'completed' row if available)
mip_data_deduped = mip_data_sorted.drop_duplicates(subset='School ID', keep='last')

mip_data_sorted['MIP_Status'].value_counts()

,count
MIP_Status,
submitted,18416
started,5701


In [10]:
# Convert school id to strings in both dataset. Merge the mip data with school data using School Id . Show how many mere merged and how many didnot merge. Replace mip status with notstarted for the unmerged observations

# Convert 'School ID' to string in both DataFrames
mip_data_deduped['School ID'] = mip_data_deduped['School ID'].astype(str)
school_data['School ID'] = school_data['School ID'].astype(str)

# Merge the two datasets
merged_data = pd.merge(school_data, mip_data_deduped,  on='School ID', how='left', indicator=True)

# Calculate the number of merged and unmerged observations
merged_count = len(merged_data[merged_data['_merge'] == 'both'])
unmerged_count = len(merged_data[merged_data['_merge'] == 'left_only'])

print(f"Number of observations merged: {merged_count}")
print(f"Number of observations not merged: {unmerged_count}")

# Replace 'MIP_Status' with 'notstarted' for unmerged observations
merged_data.loc[merged_data['_merge'] == 'left_only', 'MIP_Status'] = 'notstarted'

# Remove the '_merge' indicator column
merged_data = merged_data.drop('_merge', axis=1)

print(merged_data.head())

print(merged_data[(merged_data['MIP_Status'] != 'submitted') & (merged_data['DISTRICT NAME'] == 'VAISHALI')])

Number of observations merged: 18571
Number of observations not merged: 10416
  DISTRICT NAME BLOCK NAME    School ID           SCHOOL NAME  \
0        ARARIA     ARARIA  10071200101          UMS ARARBARI   
1        ARARIA     ARARIA  10071200301     UMS KASAILA  GIRL   
2        ARARIA     ARARIA  10071200401              MS JAMUA   
3        ARARIA     ARARIA  10071200402  U.M.S JAMUA KHAMGARA   
4        ARARIA     ARARIA  10071200403    UMS JAMUA GHARWARI   

                                   UUID      User Type User sub type  \
0                                   NaN            NaN           NaN   
1                                   NaN            NaN           NaN   
2  ee1fb08e-74ba-4647-868a-a8fb41371b80  administrator            AT   
3                                   NaN            NaN           NaN   
4                                   NaN            NaN           NaN   

  Declared State District   Block  ...                Project ID  \
0            NaN      NaN     

<ipython-input-10-d66a7b167ae3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mip_data_deduped['School ID'] = mip_data_deduped['School ID'].astype(str)


# Tables

In [11]:
#Show number of schools for mip status. Show Number of schools for each category of mip status  by district.

# Group by 'MIP_Status' and count the number of schools
mip_status_counts = merged_data.groupby('MIP_Status')['School ID'].count()
print("\nNumber of schools for each MIP status:")
print(mip_status_counts)





Number of schools for each MIP status:
MIP_Status
notstarted    10416
started        2896
submitted     15675
Name: School ID, dtype: int64


In [12]:
##### District wise Tables #####

# Calculate total number of schools per district
district_totals = merged_data.groupby('DISTRICT NAME')['School ID'].count().reset_index(name='Total Schools')

# Calculate the number of schools in each MIP status category by district
mip_by_district = merged_data.groupby(['DISTRICT NAME', 'MIP_Status'])['School ID'].count().reset_index(name='School Count')

# Merge the total schools and school counts
district_summary = pd.merge(district_totals, mip_by_district, on='DISTRICT NAME')

# Calculate the percentage of schools in each MIP status category
district_summary['Percentage'] = ((district_summary['School Count'] / district_summary['Total Schools']) * 100).round(2)  # Round to 2 decimal po

# Pivot the table to wide format
district_summary_wide = district_summary.pivot(index='DISTRICT NAME', columns='MIP_Status', values=['School Count', 'Percentage'])

# Add total schools as a column for clarity
district_summary_wide['Total Schools'] = district_totals.set_index('DISTRICT NAME')

# Flatten the MultiIndex columns
district_summary_wide.columns = [' '.join(col).strip() for col in district_summary_wide.columns.values]

# Reset the index to make the district names a column
district_summary_wide.reset_index(inplace=True)

# Display the wide-format table
print("\nWide-format Table: Number of schools (and percentage) in each MIP status category by district")
print(district_summary_wide)



Wide-format Table: Number of schools (and percentage) in each MIP status category by district
         DISTRICT NAME  School Count notstarted  School Count started  \
0               ARARIA                    397.0                  74.0   
1                ARWAL                     47.0                  27.0   
2   AURANGABAD (BIHAR)                    534.0                 119.0   
3                BANKA                    465.0                 107.0   
4            BEGUSARAI                     52.0                  37.0   
5            BHAGALPUR                    215.0                 143.0   
6              BHOJPUR                     51.0                  48.0   
7                BUXAR                    154.0                  44.0   
8            DARBHANGA                    604.0                 110.0   
9         E. CHAMPARAN                    218.0                  75.0   
10                GAYA                    641.0                 161.0   
11           GOPALGANJ       

In [13]:
# export to google sheet
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=district_summary_wide)

https://docs.google.com/spreadsheets/d/1QwKxIbDK7J-V_pWo94Yox7Rv25DsBqeDtMSV8tjNq0o#gid=0


In [14]:
#### Block-Wise Tables ####

# Calculate total number of schools per district and block
block_totals = merged_data.groupby(['DISTRICT NAME', 'BLOCK NAME'])['School ID'].count().reset_index(name='Total Schools')

# Calculate the number of schools in each MIP status category by district and block
mip_by_block = merged_data.groupby(['DISTRICT NAME', 'BLOCK NAME', 'MIP_Status'])['School ID'].count().reset_index(name='School Count')

# Merge the total schools and school counts
block_summary = pd.merge(block_totals, mip_by_block, on=['DISTRICT NAME', 'BLOCK NAME'])

# Calculate the percentage of schools in each MIP status category
block_summary['Percentage'] = ((block_summary['School Count'] / block_summary['Total Schools']) * 100).round(2)

# Pivot the table to wide format
block_summary_wide = block_summary.pivot(index=['DISTRICT NAME', 'BLOCK NAME'], columns='MIP_Status', values=['School Count', 'Percentage'])

# Add total schools as a column for clarity
block_summary_wide['Total Schools'] = block_totals.set_index(['DISTRICT NAME', 'BLOCK NAME'])

# Flatten the MultiIndex columns
block_summary_wide.columns = [' '.join(col).strip() for col in block_summary_wide.columns.values]

# Reset the index to make the district and block names columns
block_summary_wide.reset_index(inplace=True)

# Display the wide-format table
print("\nWide-format Table: Number of schools (and percentage) in each MIP status category by district and block")
print(block_summary_wide)



Wide-format Table: Number of schools (and percentage) in each MIP status category by district and block
    DISTRICT NAME  BLOCK NAME  School Count notstarted  School Count started  \
0          ARARIA      ARARIA                     50.0                  14.0   
1          ARARIA    BHARGAMA                     35.0                   7.0   
2          ARARIA  FORBISGANJ                     43.0                  10.0   
3          ARARIA     JOKIHAT                     72.0                   5.0   
4          ARARIA  KURSAKANTA                     22.0                   6.0   
..            ...         ...                      ...                   ...   
532   W CHAMPARAN      NAUTAN                     38.0                   6.0   
533   W CHAMPARAN     PIPRASI                      9.0                   1.0   
534   W CHAMPARAN    RAMNAGAR                     35.0                   3.0   
535   W CHAMPARAN       SIKTA                     31.0                   4.0   
536   W CHAMPAR

In [15]:
# export to google sheet
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=block_summary_wide)

https://docs.google.com/spreadsheets/d/1JZNW9Dbad6QDNyYZCWH7E9muPZUWiRmWnlH7FN22w0c#gid=0


# Dashboard Creation

In [16]:
# Create a Dashboard showing the District data.

import pandas as pd
from google.colab import data_table
from google.colab import drive
from google.colab import sheets

# Assuming your data is in the 'merged_data' DataFrame from the previous code

# Display the DataFrame using data_table for interactive exploration
data_table.DataTable(merged_data)

# Example of creating a summary table (replace with your desired dashboard elements)
# This uses the district_summary_wide DataFrame that was already created
data_table.DataTable(district_summary_wide)


# Example of creating a chart (replace with your desired dashboard elements and chart type)
# Install plotly for creating interactive charts
!pip install plotly

import plotly.express as px

# Create a bar chart showing the number of schools by district and MIP status
fig = px.bar(district_summary, x='DISTRICT NAME', y='School Count', color='MIP_Status',
             title="Number of Schools by District and MIP Status",
             labels={'School Count': 'Number of Schools'})
fig.show()

# Add more charts or tables to create your dashboard as needed
# You can use other plotting libraries like matplotlib or seaborn if preferred